<a href="https://colab.research.google.com/github/gabrielsimas/rio-viagens-onibus-analytics/blob/feature%2Fcriacao-ambiente/ARTIGO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚌 MVP Engenharia de Dados - O Rio de Janeiro e os Ônibus: Entendendo o Caos!
## ***Nome:*** **Luís Gabriel Nascimento Simas**
## ***Matrícula:*** **4052025000943**

## **Objetivo**

### *O objetivo do trabalho deve ser muito bem detalhado; é um planejamento do trabalho, contendo de forma clara e objetiva o problema a ser resolvido e as perguntas de negócio a serem respondidas. Será avaliada a qualidade desta descrição.*
### *Comece pelo objetivo do seu trabalho. Antes de iniciar sua busca pelos dados, pense e descreva claramente qual problema deseja resolver com este MVP. Enumere as perguntas que deseja responder.*
### *É de extrema importância que esta etapa seja feita antes de iniciar qualquer outra etapa.*
### *Uma vez traçado o objetivo e conhecendo bem qual problema se deseja resolver, quais perguntas se deseja responder, é hora de iniciar a busca pelos dados.*
### *Não é necessário atingir todos os objetivos desenhados nesta seção. Assim, não remova perguntas as quais não se conseguiu responder. Deixe a documentação do objetivo intacta e faça uma boa discussão do atingimento deste ao final do trabalho (vide Autoavaliação).*


### O Rio de Janeiro, mundialmente conhecido por suas belezas naturais, enfrenta um desafio urbano de proporções continentais: a mobilidade pública. O sistema de transporte rodoviário, vital para o funcionamento da metrópole, é frequentemente associado à palavra "**Caos**".
### A população carioca convive diariamente com incertezas: tempos de espera imprevisíveis, frota envelhecida, desconforto térmico em uma cidade tropical e falhas na cobertura do serviço, especialmente em horários noturnos. Embora iniciativas como o BRT (*Bus Rapid Transit*) e o BRS (*Bus Rapid System*) tenham sido implementadas para mitigar a lentidão, a percepção de qualidade do serviço permanece aquém do ideal.
### Não se trata apenas de desconforto, mas de um problema sistêmico que envolve a gestão de consórcios privados, a fiscalização do poder público e o impacto direto na qualidade de vida e produtividade do cidadão.
### Se por um lado o cenário físico é caótico, o cenário digital oferece uma oportunidade de organização e entendimento. A Prefeitura da Cidade do Rio de Janeiro (PCRJ), através do portal *[data.rio](https://data.rio)*, disponibiliza um vasto repositório de dados abertos em vários formatos e alguns até como tabelas no **Google** *BigQuery*.
### Esses dados incluem registros detalhados de viagens (baseados em bilhetagem e telemetria consolidada), cadastro técnico da frota, dados meteorológicos do Alerta Rio e registros de reclamações da central 1746.

### Para guiar o desenvolvimento do pipeline de dados, definimos as seguintes questões-chave que esse trabalho deseja responder.


- ### **Dinâmica de Viagens e Demanda**
    - #### 1. Em quais dias e faixas horárias o volume de viagens realizadas atinge seu pico?
    - #### 2. Existe uma redução sensível na oferta de viagens nos finais de semana comparados aos dias úteis?
    - #### 3. Qual é a duração média das viagens por Consórcio? Existe discrepância significativa entre eles?

- ### **Consistência Operacional**
    - #### 4. O tempo de viagem nas principais linhas é consistente ou apresenta alto desvio padrão (imprevisibilidade)?
    - #### 5. Em dias classificados como "Chuvosos" (precipitação média acima de um limiar), observa-se aumento no tempo médio de deslocamento?
    - #### 6. A chuva influencia na quantidade total de viagens realizadas (redução da oferta ou demanda)?

- ### Perfil da frota e qualidade
    - #### 7. Qual é a idade média da frota de ônibus ativa no Rio de Janeiro?
    - #### 8. Quais consórcios operam com a frota mais moderna e quais possuem a frota mais obsoleta?
    - #### 9. Existe correlação entre a idade da frota e a presença de ar-condicionado funcional?

- ### Percepção do Usuário (1746)
    - #### 10. Quais são as categorias de reclamação mais frequentes relacionadas ao transporte rodoviário?

### **Escopo temporal**
#### A análise compreenderá o período de **01/01/2024** a **19/12/2025**, permitindo uma visão longitudinal que abrange sazonalidades, dias úteis, feriados e variações climáticas ao longo de quase dois anos completos.
#### Antes de começar a explicar o mecanismo de coleta de dados que foi utilizado, é importante mostrar em qual arquitetura nosso trabalho está sendo desenvolvido/implementado.


## **Arquitetura**

### Decidi sair um pouco do **Databricks** para utilizar o que há de mais moderno em voga no mercado. Saí de uma solução proprietária para uma solução cloud native utilizando algumas ferramentas open-source.
### Aqui, vou utilizar o que chamamos de **Arquitetura Modernas de Dados**

#### O que é a **Arquitetura Moderna de Dados**
##### É uma arquitetura **flexível**, **escalável** e **baseada na nuvem**. Ela surge dos modelos tradicionais para suportar o volume e a velocidade dos dados atuais, integrando **Big Data**, **IA** e **governança**. Ela tem como características principais:
- ##### **Nuvem** e **Tempo Real**: Prioriza a elasticidade (AWS, Azure, GCP) e a disponibilidade imediata dos dados para tomadas de decisão rápidas. Neste MVP, vamos utilizar a GCP como nuvem principal, mas a solução que vamos implementar é totalmente agnóstica em questões de hospedagem.
- ##### **Dados como Produto**: O gerenciamento é focado no valor que o dado gera para o negócio.
- ##### **Governança e Segurança**:  Implementadas na base (considerando LGPD), com forte gestão de metadados e interoperabilidade via APIs.
##### As abordagens mais populares dessa arquitetura são:
- ##### **Data Lakehouse**: O melhor dos dois mundos (flexibilidade do Lake + confiabilidade do Warehouse). ***Será a arquitetura que utilizaremos nesse MVP.***
- ##### **Data Mesh**: Descentraliza a propriedade dos dados por domínios de negócio.
- ##### **Modern Data Stack (MDS)**: Orquestração de diversas ferramentas especializadas.

### A **Arquitetura moderna de dados** tem como principal objetivo unificar dados, análises e cargas de trabalho de Inteligência Artificial de forma **eficiente, confiável e escalável**.
### ***Nesse MVP, vamos focar no uso do Data Lakehouse com a Arquitetura Medalhão***
### Antes de irmos para a infraestrutura utilizada para suportar essa Arquitetura, passaremos rapidamente sobre os significados e propósitos do **Data Lakehouse** e **Arquitetura Medalhão**


### **Data Lakehouse**
<center>
<img src="https://drive.google.com/uc?id=1jV69fy2AMAd4YZx0BACaQPwHQLkCJNH1" alt="Girl in a jacket" width="400">
<center />



## **Base de Dados alvo**

## **Coleta dos dados**

## **Modelagem**

## **Carga**

## **Análise**

### **Qualidade dos dados**

### **Solução do Problema**